[View in Colaboratory](https://colab.research.google.com/github/GeniGaus/MLBlr/blob/master/DNST.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.8
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 33s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
BatchNormInit = BatchNormalization()(input)
reluInit = Activation('relu')(BatchNormInit)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(reluInit)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 3)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         activation_1[0][0]               
__________________________________________________________________________________________________
batch_norm

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
43776/50000 [=========================>....] - ETA: 35s - loss: 1.7322 - acc: 0.3581

50000/50000 [==============================] - 298s 6ms/step - loss: 1.7069 - acc: 0.3685 - val_loss: 1.9469 - val_acc: 0.3692
Epoch 2/50
26240/50000 [==============>...............] - ETA: 2:09 - loss: 1.4506 - acc: 0.4693

50000/50000 [==============================] - 286s 6ms/step - loss: 1.3866 - acc: 0.4945 - val_loss: 1.4069 - val_acc: 0.4969
Epoch 3/50
19712/50000 [==========>...................] - ETA: 2:45 - loss: 1.1844 - acc: 0.5723

50000/50000 [==============================] - 287s 6ms/step - loss: 1.1401 - acc: 0.5895 - val_loss: 1.2861 - val_acc: 0.5600
Epoch 4/50
17088/50000 [=========>....................] - ETA: 3:00 - loss: 1.0373 - acc: 0.6275

50000/50000 [==============================] - 287s 6ms/step - loss: 1.0157 - acc: 0.6351 - val_loss: 1.1489 - val_acc: 0.6136
Epoch 5/50
16064/50000 [========>.....................] - ETA: 3:06 - loss: 0.9518 - acc: 0.6602

...

50000/50000 [==============================] - 287s 6ms/step - loss: 0.3457 - acc: 0.8768 - val_loss: 0.6051 - val_acc: 0.8231
Epoch 47/50
15360/50000 [========>.....................] - ETA: 3:09 - loss: 0.3374 - acc: 0.8805

50000/50000 [==============================] - 287s 6ms/step - loss: 0.3422 - acc: 0.8777 - val_loss: 0.6151 - val_acc: 0.8237
Epoch 48/50
15360/50000 [========>.....................] - ETA: 3:10 - loss: 0.3334 - acc: 0.8824

50000/50000 [==============================] - 287s 6ms/step - loss: 0.3379 - acc: 0.8803 - val_loss: 0.6459 - val_acc: 0.8173
Epoch 49/50
15360/50000 [========>.....................] - ETA: 3:10 - loss: 0.3259 - acc: 0.8835

50000/50000 [==============================] - 287s 6ms/step - loss: 0.3357 - acc: 0.8826 - val_loss: 0.7177 - val_acc: 0.7928
Epoch 50/50
15360/50000 [========>.....................] - ETA: 3:10 - loss: 0.3194 - acc: 0.8870

50000/50000 [==============================] - 288s 6ms/step - loss: 0.3295 - acc: 0.8834 - val_loss: 0.6884 - val_acc: 0.8129


In [16]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 18s 2ms/step
Test loss: 0.6883799523353576
Test accuracy: 0.8129


In [14]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_4_bs64_dropout02_cmp08.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')

In [15]:
# Saving using PyDrive to google drive

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'DNST_model_4_bs64_dropout02_cmp08.h5'})
uploaded.SetContentFile('DNST_model_4_bs64_dropout02_cmp08.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1Dn6zcz_XUTvmY1VLIRaXthUC_E-KgvHU
